In [6]:
import json
import torch
import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [8]:
dataset = 'nontoxic'  # Or toxic
model = 'mistral'  # change model name

classifier_id = "nicholasKluge/ToxicityModel"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(classifier_id)
toxicity_model = AutoModelForSequenceClassification.from_pretrained(classifier_id)
toxicity_model.eval()
toxicity_model.to(device)


objs = []
with open(f"{model}/data_out/{dataset}_out.jsonl") as f:
    for line in tqdm.tqdm(f):
        obj = json.loads(line)
        if "toxicity_score" not in obj["prompt"]:
            prompt = obj["prompt"]["text"]
            tokens = tokenizer(prompt, return_tensors="pt", max_length=512)
            tokens.to(device)
            score = toxicity_model(**tokens)[0].item()
            obj["prompt"]["toxicity_score"] = score

            cont = obj["mistralai/Mistral-7B-Instruct-v0.2_response_with_system_prompt"]  # change this one
            tokens = tokenizer(cont, return_tensors="pt", max_length=512)
            tokens.to(device)
            score = toxicity_model(**tokens)[0].item()
            obj[f"{model}-toxicity_score_with_system_prompt"] = score
            
            cont = obj["mistralai/Mistral-7B-Instruct-v0.2_response"]  # change this one
            tokens = tokenizer(cont, return_tensors="pt", max_length=512)
            tokens.to(device)
            score = toxicity_model(**tokens)[0].item()
            obj[f"{model}-toxicity_score"] = score

        objs.append(obj)

with open(f"{model}/data_out/{dataset}_out2.jsonl", "w") as f:
    for obj in objs:
        f.write(f"{json.dumps(obj)}\n")

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1200it [02:37,  7.64it/s]
